In [1]:
!pip3 install pandas scikit-learn

     |████████████████████████████████| 12.2 MB 222 kB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 499 kB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 15.4 MB/s eta 0:00:01     |█████████████████▍              | 18.8 MB 13.0 MB/s eta 0:00:02     |███████████████████████▌        | 25.4 MB 13.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Web attack detection using CNN-BiLSTM neural network and CICIDS2017 dataset

In this notebook we implement web attack detection using CNN-BiLSTM neural network and CICIDS2017 dataset, and compare this neural network performance to our previously developed Random Forest model (https://github.com/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/web-attack-detection.ipynb).

Training data: https://github.com/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/datasets/web_attacks_balanced.zip

Training dataset is the balanced dataset based on CICIDS2017: https://www.unb.ca/cic/datasets/ids-2017.html
    
Sources:
- https://paperswithcode.com/paper/efficient-deep-cnn-bilstm-model-for-network
- Jay Sinha and M. Manollas. 2020. Efficient Deep CNN-BiLSTM Model for Network Intrusion Detection. In Proceedings of the 2020 3rd International Conference on Artificial Intelligence and Pattern Recognition (AIPR 2020). Association for Computing Machinery, New York, NY, USA, 223–231. DOI:https://doi.org/10.1145/3430199.3430224.
- https://github.com/razor08/Efficient-CNN-BiLSTM-for-Network-IDS/blob/master/NSL_KDD_Final.ipynb
    

## Data preprocessing

In [2]:
from itertools import cycle
import pickle
import sys
import time
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Convolution1D, MaxPooling1D, Reshape
from tensorflow.keras.models import Sequential

%matplotlib inline

Download the dataset from Github to Google Colab and unzip it.

Load our dataset and separate it into feature vectors and labels. As explained in the previous work, 7 features (Flow ID, Source IP, Source Port, Destination IP, Destination Port, Protocol, Timestamp) are excluded from the dataset (see https://github.com/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/web-attack-detection.ipynb).

In [3]:
data = pd.read_csv('../dataset/web_attacks_balanced.csv', delimiter=',')

y = data['Label']
X = data.drop(labels=['Label', 'Flow ID', 'Source IP', 'Source Port',
              'Destination IP', 'Destination Port', 'Protocol', 'Timestamp'], axis=1)

### Preparing feature vectors for training

First, we need to inspect feature vectors.

In [4]:
X

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,350.0,4.0,4.0,116.0,116.0,29.0,29.0,29.000000,0.000000,29.0,...,3.0,32.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
1,203.0,2.0,2.0,88.0,188.0,44.0,44.0,44.000000,0.000000,94.0,...,1.0,32.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
2,159.0,2.0,2.0,102.0,224.0,51.0,51.0,51.000000,0.000000,112.0,...,1.0,20.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
3,179.0,2.0,2.0,80.0,180.0,40.0,40.0,40.000000,0.000000,90.0,...,1.0,32.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
4,199.0,2.0,2.0,88.0,188.0,44.0,44.0,44.000000,0.000000,94.0,...,1.0,32.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,23405.0,1.0,1.0,45.0,103.0,45.0,45.0,45.000000,0.000000,103.0,...,0.0,32.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
7263,23885.0,2.0,2.0,76.0,184.0,38.0,38.0,38.000000,0.000000,92.0,...,1.0,20.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
7264,8269.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,...,1.0,20.0,0.00000,0.00000,0.0,0.0,0.000,0.00000,0.0,0.0
7265,91853480.0,13.0,13.0,496.0,2064.0,430.0,0.0,38.153846,117.746795,2052.0,...,12.0,20.0,41165.77778,12795.61076,75286.0,36806.0,9994317.222,3713.02605,10000000.0,9989268.0


Check the absence of missing values in feature vectors.

In [5]:
X.isnull().values.any()

False

As we can see, the range of features is not normalized. We need to perform feature scaling for the correct functioning of selected deep learning algorithms (CNN and BiLSTM).

In our case, the min-max normalization is used: all selected features are transformed into the range [0, 1].

In [6]:
scaler = sklearn.preprocessing.MinMaxScaler()

To prevent data leakage, feature scaling would be performed on cross-validation folds at the training step.

### Preparing labels for training

First, we need to inspect labels.

In [7]:
y

0       BENIGN
1       BENIGN
2       BENIGN
3       BENIGN
4       BENIGN
         ...  
7262    BENIGN
7263    BENIGN
7264    BENIGN
7265    BENIGN
7266    BENIGN
Name: Label, Length: 7267, dtype: object

Check the absence of missing values in labels.

In [8]:
y.isnull().values.any()

False

As we can see, labels are categorical. We would need to transform them into the numeric form before the neural network training. In order to achieve that, we would apply one-hot encoding to labels.

To prevent data leakage, one-hot encoding would be performed within cross-validation folds at the training step.

## The neural network preparation

Define and compile the neural network.
<p> We use a sequential model with CNN and BiLSTM layers, and batch normalization is applied before BiLSTM layers. A dropout layer is put before the final layer to prevent overfitting.
<p> Categorical cross-entropy is chosen as a loss function because of the multiclass classification. To get the classification result, the softmax activation function is applied to the output of the last, fully connected layer. 

In [9]:
batch_size = 32
model = Sequential()
model.add(Convolution1D(64, kernel_size=32, padding="same",
          activation="relu", input_shape=(76, 1)))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Reshape((128, 1), input_shape=(128, )))

model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=False)))

model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary(line_length=100)

Model: "sequential"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 conv1d (Conv1D)                             (None, 76, 64)                          2112           
                                                                                                    
 max_pooling1d (MaxPooling1D)                (None, 15, 64)                          0              
                                                                                                    
 batch_normalization (BatchNormalization)    (None, 15, 64)                          256            
                                                                                                    
 bidirectional (Bidirectional)               (None, 128)                             66048          
                                                                       

## The neural network training

We use Stratified K-Folds cross-validation in our experiment.

In [11]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Evaluation metrics are stored for each fold.

In [12]:
scores = pd.DataFrame(columns=[1, 2, 3, 4, 5, 'Mean'],
                      index=['Accuracy', 'Precision', 'Recall', 'F1'])

We use the following function to get evaluation metrics.

In [13]:
def print_metrics(y_eval: np.ndarray, y_pred: np.ndarray, average: str = 'binary') -> List[float]:
    accuracy = metrics.accuracy_score(y_eval, y_pred)
    precision = metrics.precision_score(y_eval, y_pred, average=average)
    recall = metrics.recall_score(y_eval, y_pred, average=average)
    f1 = metrics.f1_score(y_eval, y_pred, average=average)

    print('Accuracy =', accuracy)
    print('Precision =', precision)
    print('Recall =', recall)
    print('F1 =', f1)

    return [accuracy, precision, recall, f1]

All test and predicted labels are also stored at each step of cross-validation. They will be used to calculate a confusion matrix.

In [14]:
actual_targets = np.empty([0], dtype=float)
predicted_targets = np.empty([0], dtype=float)

On each fold, we use the following function to transform the data, to fit and to validate the neural network. Note that data preprocessing is largely performed at the cross-validation step to prevent data leakage.

In [15]:
def train_and_validate(model: Sequential,
                       X: pd.DataFrame,
                       y: pd.Series,
                       train_index: np.ndarray,
                       test_index: np.ndarray,
                       scaler: sklearn.preprocessing.MinMaxScaler) -> Tuple[np.ndarray, np.ndarray]:
    # Get the test data and the validation data for a current fold.
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print("Train index:", train_index)
    print("Test index:", test_index)

    # Feature scaling of data.
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

    # We need to reshape feature vectors to use them as the neural network input.
    X_train = np.reshape(
        X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(
        X_test, (X_test.shape[0], X_test.shape[1], 1))

    # One-Hot encoding for labels.
    y_train_encoded = pd.get_dummies(y_train).values
    y_test_encoded = pd.get_dummies(y_test).values

    # Fit the neural network model.
    model.fit(X_train, y_train_encoded, validation_data=(
        X_test, y_test_encoded), epochs=10)

    # Apply the neural network model to the validation data and get the accuracy of this prediction.
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_eval = np.argmax(y_test_encoded, axis=1)

    return y_eval, y_pred

The training step is designed for the use with different versions of Tensorflow, if necessary. It is also possible to choose whether the training should be performed by CPU or GPU.

This step may take some time, 13 minutes or more depending on the CPU performance (it will be faster with GPU).

In [16]:
# Tensorflow 1.14.

# This option may be used to speed up the CPU computation in some cases.
# import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_cpu_global_jit'

# Configure session parameters for the optimal parallelization and choose the computational device (CPU, GPU or both).
# config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=48,
#                                   inter_op_parallelism_threads=48, allow_soft_placement=True, device_count={'CPU': 24})
# with tf.compat.v1.Session(config=config) as sess:

# Tensorflow 2.X, the computation is performed on CPU.
# with tf.device('/CPU:0'):

# Tensorflow 2.X, the computation is performed on GPU.
with tf.device('/GPU:0'):

    # Set a timer for the learning time evaluation.
    start_time = time.time()
    
    i = 1
    # Stratified K-Folds cross-validation.
    for train_index, test_index in cv.split(X, y):
        y_eval, y_pred = train_and_validate(
            model, X, y, train_index, test_index, scaler)
        # Store test and predicted labels.
        actual_targets = np.append(actual_targets, y_eval)
        predicted_targets = np.append(predicted_targets, y_pred)
        # Get and print validation scores.
        print("Validation scores:")
        scores[i] = print_metrics(y_eval, y_pred, average='weighted')
        i += 1

    # Get the learning time.
    print('Total operation time: ', time.time() - start_time, 'seconds')

Train index: [   1    3    4 ... 7262 7264 7265]
Test index: [   0    2   17 ... 7258 7263 7266]
Epoch 1/10
182/182 [==============================] - 17s 31ms/step - loss: 0.3901 - accuracy: 0.8476 - val_loss: 0.8217 - val_accuracy: 0.7001
Epoch 2/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2985 - accuracy: 0.8696 - val_loss: 1.2330 - val_accuracy: 0.5062
Epoch 3/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2842 - accuracy: 0.8746 - val_loss: 0.3811 - val_accuracy: 0.8549
Epoch 4/10
182/182 [==============================] - 5s 26ms/step - loss: 0.2919 - accuracy: 0.8713 - val_loss: 0.2890 - val_accuracy: 0.8721
Epoch 5/10
182/182 [==============================] - 4s 24ms/step - loss: 0.2822 - accuracy: 0.8749 - val_loss: 0.3202 - val_accuracy: 0.8611
Epoch 6/10
182/182 [==============================] - 5s 26ms/step - loss: 0.2669 - accuracy: 0.8760 - val_loss: 0.3540 - val_accuracy: 0.8707
Epoch 7/10
182/182 [========================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


182/182 [==============================] - 5s 26ms/step - loss: 0.2547 - accuracy: 0.8853 - val_loss: 0.2489 - val_accuracy: 0.8865
Epoch 2/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2615 - accuracy: 0.8798 - val_loss: 0.2408 - val_accuracy: 0.8838
Epoch 3/10
182/182 [==============================] - 4s 24ms/step - loss: 0.2593 - accuracy: 0.8810 - val_loss: 0.2711 - val_accuracy: 0.8659
Epoch 4/10
182/182 [==============================] - 4s 24ms/step - loss: 0.2504 - accuracy: 0.8846 - val_loss: 0.2644 - val_accuracy: 0.8851
Epoch 5/10
182/182 [==============================] - 4s 25ms/step - loss: 0.2503 - accuracy: 0.8816 - val_loss: 0.2534 - val_accuracy: 0.8831
Epoch 6/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2483 - accuracy: 0.8854 - val_loss: 0.2871 - val_accuracy: 0.8817
Epoch 7/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2588 - accuracy: 0.8842 - val_loss: 0.2640 - val_accuracy: 0.8824
Epoch 8/10

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


182/182 [==============================] - 5s 27ms/step - loss: 0.2437 - accuracy: 0.8875 - val_loss: 0.3339 - val_accuracy: 0.8217
Epoch 2/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2396 - accuracy: 0.8880 - val_loss: 0.2826 - val_accuracy: 0.8789
Epoch 3/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2374 - accuracy: 0.8889 - val_loss: 0.2685 - val_accuracy: 0.8768
Epoch 4/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2370 - accuracy: 0.8906 - val_loss: 0.2827 - val_accuracy: 0.8796
Epoch 5/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2332 - accuracy: 0.8913 - val_loss: 0.2583 - val_accuracy: 0.8789
Epoch 6/10
182/182 [==============================] - 4s 24ms/step - loss: 0.2327 - accuracy: 0.8927 - val_loss: 0.2706 - val_accuracy: 0.8809
Epoch 7/10
182/182 [==============================] - 5s 25ms/step - loss: 0.2374 - accuracy: 0.8872 - val_loss: 0.2716 - val_accuracy: 0.8796
Epoch 8/10

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model evaluation

Here we can see evaluation metrics for each fold and their mean values.

In [17]:
scores['Mean'] = scores.mean(axis=1)
scores

,1,2,3,4,5,Mean
Accuracy,0.863136,0.886520,0.712319,0.886442,0.878871,0.845458
Precision,0.789593,0.877003,0.799120,0.905784,0.811065,0.836513
Recall,0.863136,0.886520,0.712319,0.886442,0.878871,0.845458
F1,0.824097,0.851096,0.607140,0.851048,0.841153,0.794907


Visualize a confusion matrix for all folds of data to evaluate the classifier's performance. Here and further we use the following function.

In [18]:
def plot_confusion_matrix(y_eval: np.ndarray, y_pred: np.ndarray, labels: List[str]) -> None:
    # Get the confusion matrix for the last fold of data.
    confusion_matrix = metrics.confusion_matrix(y_eval, y_pred)

    # Evaluate accuracy and misclassification scores.
    accuracy = np.trace(confusion_matrix) / float(np.sum(confusion_matrix))
    misclass = 1 - accuracy

    # Plot the confusion matrix.
    metrics.ConfusionMatrixDisplay.from_predictions(
        y_eval, y_pred, display_labels=labels,
        xticks_rotation='vertical', cmap='Blues')
    plt.xlabel('Predicted label\n\naccuracy={:0.4f}; misclass={:0.4f}'.format(
        accuracy, misclass))
    plt.title('Confusion Matrix')
    plt.show()

In [19]:
plot_confusion_matrix(actual_targets, predicted_targets, [
                      'BENIGN', 'Web Attack - Brute Force', 'Web Attack - Sql Injection', 'Web Attack - XSS'])

From the above confusion matrix, we can conclude that the model is mostly adequate in identifying benign traffic and brute force attacks.

## model save

In [20]:
model.save('./model/webattack_detection_rf_model')

INFO:tensorflow:Assets written to: ./model/webattack_detection_rf_model/assets


INFO:tensorflow:Assets written to: ./model/webattack_detection_rf_model/assets


In [29]:
pre = model.predict(X)

228/228 [==============================] - 2s 8ms/step


In [33]:
pre

array([[9.35673952e-01, 9.28300340e-03, 2.78573111e-03, 5.22573255e-02],
       [9.91657376e-01, 1.36062631e-03, 6.49646681e-05, 6.91705011e-03],
       [9.86032665e-01, 1.06245931e-03, 6.16583158e-04, 1.22882500e-02],
       ...,
       [9.99660015e-01, 1.33794529e-04, 1.09597377e-05, 1.95152257e-04],
       [9.88495588e-01, 7.49930087e-03, 5.92261029e-04, 3.41280969e-03],
       [9.88648593e-01, 1.18403614e-03, 1.33840425e-04, 1.00335535e-02]],
      dtype=float32)

In [32]:
pre[0]

array([0.93567395, 0.009283  , 0.00278573, 0.05225733], dtype=float32)

## Comparison to Random Forest model

Download and open our previously developed model for this dataset: Random Forest (see https://colab.research.google.com/github/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/web-attack-detection.ipynb).

In [21]:
with open('../Random Forest/model/webattack_detection_rf_model.pkl', 'rb') as f:
    rfc = pickle.load(f)
rfc

RandomForestClassifier(max_depth=17, max_features=10, min_samples_leaf=3,
                       n_estimators=50, oob_score=True, random_state=42)

We use the indices of the final fold of Stratified K-Folds cross-validation to split the dataset into a training set with 80% of all the samples and test set with 20% of all the samples, preserving the percentage of samples for each class.

Our Random Forest model does not need feature scaling and uses only selected features, so we need to get correct feature vectors.

In [22]:
columns = ['Average Packet Size',
           'Flow Bytes/s',
           'Max Packet Length',
           'Fwd Packet Length Mean',
           'Fwd IAT Min',
           'Total Length of Fwd Packets',
           'Flow IAT Mean',
           'Fwd Packet Length Max',
           'Fwd IAT Std',
           'Fwd Header Length']
X_test = X.iloc[test_index]
X_test = X_test[columns]
X_test

,Average Packet Size,Flow Bytes/s,Max Packet Length,Fwd Packet Length Mean,Fwd IAT Min,Total Length of Fwd Packets,Flow IAT Mean,Fwd Packet Length Max,Fwd IAT Std,Fwd Header Length
5,7.000000,7.000000e+06,7.0,3.500,1.0,7.0,1.000000e+00,7.0,0.000000e+00,64.0
7,51.000000,1.037378e+01,48.0,48.000,1996301.0,384.0,4.935522e+06,48.0,2.140000e+07,140.0
8,48.000000,1.045752e+06,48.0,32.000,4.0,64.0,5.100000e+01,32.0,0.000000e+00,64.0
10,82.181818,4.116763e+04,236.0,70.000,4.0,490.0,2.195900e+03,236.0,7.165981e+03,164.0
11,7.200000,1.071429e+06,6.0,6.000,0.0,6.0,7.000000e+00,6.0,0.000000e+00,20.0
...,...,...,...,...,...,...,...,...,...,...
7248,291.166667,1.211840e+04,1485.0,67.875,3.0,543.0,2.621109e+04,190.0,6.239530e+04,172.0
7249,320.500000,9.934265e+02,1460.0,46.125,2.0,369.0,3.441288e+05,194.0,1.786676e+06,184.0
7251,145.500000,5.200956e+03,201.0,45.000,0.0,45.0,4.729900e+04,45.0,0.000000e+00,32.0
7253,88.000000,2.854464e+03,98.0,39.000,0.0,39.0,4.799500e+04,39.0,0.000000e+00,20.0


Transform categorical labels into numeric form with simple label encoding. Note that our Random Forest model was developed for a binary classification task, so we transform labels accordingly: "0" indicates a benign instance and "1" an attack.

In [23]:
y_test = y[test_index]
y_test = y_test.apply(lambda x: 0 if x == 'BENIGN' else 1).values

Get the Random Forest model's evaluation metrics for the final fold of data.

Accuracy = 0.9435650378527185
Precision = 0.9214285714285714
Recall = 0.8876146788990825
F1 = 0.9042056074766354


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
y_pred_rfc = rfc.predict(X_test.values)

metrics_rfc = print_metrics(y_test, y_pred_rfc)
plot_confusion_matrix(y_test, y_pred_rfc, ['BENIGN', 'Web Attack'])

The Random Forest model is designed for binary classification, whereas the CNN-BiLSTM model is designed for multiclass classification. To compare two models correctly, we change the output shape of the neural network model's dense layer so that the neural network predicts only two classes.

In [23]:
batch_size = 32
model = Sequential()
model.add(Convolution1D(64, kernel_size=32, padding="same",
          activation="relu", input_shape=(76, 1)))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Reshape((128, 1), input_shape=(128, )))

model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=False)))

model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [24]:
model.summary(line_length=100)

Model: "sequential_1"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
conv1d_1 (Conv1D)                            (None, 76, 64)                          2112           
____________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)               (None, 15, 64)                          0              
____________________________________________________________________________________________________
batch_normalization_2 (BatchNormalization)   (None, 15, 64)                          256            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)              (None, 128)                             66048          
_____________________________________________________________________

Train the modified neural network on the final fold of data.

In [25]:
# Transform categorical labels into numeric form.
y = data['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1).values

y_eval, y_pred = train_and_validate(
    model, X, y, train_index, test_index, scaler)

Train index: [   0    1    2 ... 7264 7265 7266]
Test index: [   5    7    8 ... 7251 7253 7262]
Epoch 1/10
182/182 [==============================] - 31s 99ms/step - loss: 0.1601 - accuracy: 0.9384 - val_loss: 0.5653 - val_accuracy: 0.8513
Epoch 2/10
182/182 [==============================] - 16s 86ms/step - loss: 0.1034 - accuracy: 0.9589 - val_loss: 0.6786 - val_accuracy: 0.6676
Epoch 3/10
182/182 [==============================] - 15s 84ms/step - loss: 0.1307 - accuracy: 0.9501 - val_loss: 0.1301 - val_accuracy: 0.9436
Epoch 4/10
182/182 [==============================] - 15s 84ms/step - loss: 0.0923 - accuracy: 0.9628 - val_loss: 0.1336 - val_accuracy: 0.9443
Epoch 5/10
182/182 [==============================] - 15s 85ms/step - loss: 0.0929 - accuracy: 0.9628 - val_loss: 0.1074 - val_accuracy: 0.9484
Epoch 6/10
182/182 [==============================] - 16s 88ms/step - loss: 0.0843 - accuracy: 0.9635 - val_loss: 0.0876 - val_accuracy: 0.9429
Epoch 7/10
182/182 [===================

Get the CNN-BiLSTM model's evaluation metrics for binary classification.

Accuracy = 0.9476944253269098
Precision = 0.9591836734693877
Recall = 0.8623853211009175
F1 = 0.9082125603864735


In [26]:
metrics_nn = print_metrics(y_eval, y_pred)
plot_confusion_matrix(y_eval, y_pred, ['BENIGN', 'Web Attack'])

Finally, compare evaluation metrics for Random Forest and CNN-BiLSTM models. Here we can see that the Random Forest model shows better results than the CNN-BiLSTM model, but the latter shows good results none the less. Further research is needed to achieve the desired performance.

In [27]:
# Visualizing the data with a multiple bar chart.

# Scale the plot.
plt.figure(figsize=[15, 9])
plt.gca().set_ylim(0.8, 1)

# Get the data for the plot.
comparison_frame = pd.DataFrame(
    {'CNN-BiLSTM': metrics_nn, 'Random Forest': metrics_rfc},
    index=['Accuracy', 'Precision', 'Recall', 'F1'])

# Set the width of the bars.
wd = 0.3
x_pos = np.arange(1, 2*len(comparison_frame), 2)

# Plotting the multiple bar graphs on the same figure.
plt.bar(x_pos, comparison_frame['CNN-BiLSTM'], color='r', width=wd, edgecolor='k',
        label='CNN-BiLSTM')
plt.bar(x_pos+wd, comparison_frame['Random Forest'], color='y', width=wd, edgecolor='k',
        label='Random Forest')

# Add ticks, a title, and a legend.
plt.xticks(x_pos+wd, comparison_frame.index.values, fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Metric', fontsize=17)
plt.ylabel('Value', fontsize=17)
plt.title(
    "Performance comparison of CNN-BiLSTM and Random Forest models.", fontsize=20)
plt.legend(loc='best', fontsize=12)

plt.show()